


# Document Similarity with Latent Semantic Analysis (LSA)

The following notebook walks you through doing LSA document similarity in Python. We then output the document similarity matrix as a .csv file which can be manipulated to highlight similarity between documents. We also output a heatmap which gives an initial impression of the similarity between documents. 

###  Before we begin
Before we start, you will need to have set up a [Carbonate account](https://kb.iu.edu/d/aolp) in order to access [Research Desktop (ReD)](https://kb.iu.edu/d/apum). You will also need to have access to ReD through the [thinlinc client](https://kb.iu.edu/d/aput). If you have not done any of this, or have only done some of this, but not all, you should go to our [textPrep-Py.ipynb](https://github.com/cyberdh/Text-Analysis/blob/master/Intro/Python/Py_notebooks/textPrep-Py.ipynb) before you proceed further. The textPrep-Py notebook provides information and resources on how to get a Carbonate account, how to set up ReD, and how to get started using the Jupyter Notebook on ReD.   

### Run CyberDH environment
The code in the cell below points to a Python environment specificaly for use with the Python Jupyter Notebooks created by Cyberinfrastructure for Digital Humanities. It allows for the use of the different pakcages in our notebooks and their subsequent data sets.

##### Packages
- **sys:** Provides access to some variables used or maintained by the interpreter and to functions that interact strongly with the interpreter. It is always available.
- **os:** Provides a portable way of using operating system dependent functionality.

#### NOTE: This cell is only for use with Research Desktop. You will get an error if you try to run this cell on your personal device!!

In [1]:
import sys
import os
sys.path.insert(0,"/N/u/cyberdh/Carbonate/dhPyEnviron/lib/python3.6/site-packages")
os.environ["NLTK_DATA"] = "/N/u/cyberdh/Carbonate/dhPyEnviron/nltk_data"

### Include necessary packages for notebook 

Python's extensibility comes in large part from packages. Packages are groups of functions, data, and algorithms that allow users to easily carry out processes without recreating the wheel. Some packages are included in the basic installation of Python, others created by Python users are available for download. Make sure to have the following packages installed before beginning so that they can be accessed while running the scripts.

In your terminal, packages can be installed by simply typing `pip install nameofpackage --user`. However, since you are using ReD and our Python environment, you will not need to install any of the packages below to use this notebook. Anytime you need to make use of a package, however, you need to import it so that Python knows to look in these packages for any functions or commands you use. Below is a brief description of the packages we are using in this notebook:  

- **sklearn:** Simple and efficient tools for data mining and data analysis built on NumPy, SciPy, and matplotlib.
- **pandas:** An open source, BSD-licensed library providing high-performance, easy-to-use data structures and data analysis tools for the Python programming language.
- **warnings:** Allows for the manipulation of warning messages in Python.
- **numpy:** a general-purpose array-processing package designed to efficiently manipulate large multi-dimensional arrays of arbitrary records without sacrificing too much speed for small multi-dimensional arrays. 
- **re:** Provides regular expression matching operations similar to those found in Perl.
- **os:** Provides a portable way of using operating system dependent functionality.
- **string:** Contains a number of functions to process standard Python strings.
- **nltk:** A leading platform for building Python programs to work with human language data.
- **seaborn:** a Python data visualization library based on matplotlib. It provides a high-level interface for drawing attractive and informative statistical graphics.
- **matplotlib:** Produces publication quality 2D graphics for interactive graphing, scientific publishing, user interface development and web application servers targeting multiple user interfaces and hardcopy output formats.

In [2]:
import sklearn
# import all of the scikit learn stuff

from sklearn.decomposition import TruncatedSVD
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.preprocessing import Normalizer
from sklearn import metrics
from sklearn.cluster import KMeans, MiniBatchKMeans
from sklearn.metrics.pairwise import cosine_similarity
import pandas as pd
import warnings
import numpy as np
import re
from os.path import join, isfile, splitext
import string
import nltk
from nltk.stem import SnowballStemmer
from nltk.corpus import stopwords
import seaborn as sns
import matplotlib.pyplot as plt

This will ignore deprecation and future warnings. All the warnings in this code are not concerning and will not break the code or cause errors in the results.

In [3]:
# Suppress warnings from pandas library
warnings.filterwarnings("ignore", category=DeprecationWarning,
                        module="pandas", lineno=570)
warnings.filterwarnings("ignore", category=FutureWarning,
                        module = "sklearn", lineno = 1059)

### Getting your data

#### File paths
Here we are saving as variables different file paths that we need in our code. We do this so that they are easier to call later and so that you can make most of your changes now and not need to make as many changes later. 

First we use the `os` package above to find our `["HOME"]` directory using the `environ` function. This will work for any operating system, so if you decide to try this out on your personal computer instead of ReD, the `homePath` variable will still be the path to your 'home' directory, so no changes are needed.

Next, we combine the `homePath` variable with the folder names that lead to where our data is stored. Note that we do not use any file names yet, just the path to the folder. This is because we are comparing documents to one another, so we need to read in an entire directory. You will want to change the folder names to match your folder names in your file path.

Now we add the `homePath` variable to other folder names that lead to a folder where we will want to save any output generated by this code. You again will want to change the file names in the path to match your own file names. We save this file path as the variable `dataResults`.

In [4]:
homePath = os.environ["HOME"]
dataHome = os.path.join(homePath, "Text-Analysis-master", "data", "shakespeareDated")
dataResults = os.path.join(homePath, "Text-Analysis-master", "Output")

#### Set needed variables

Now we are setting needed variables that will help determine what the code will do farther down. Just like the file path variables, this is done so you have to make fewer changes later and also to make the changes easier to find by putting them in one place.

If you want to use the stopword list that comes with the nltk package then set `nltkStop` equal to **True**. If you do not wish to use the nltk stopword list then set `nltkStop` equal to **False**.

If you have created your own custom stopword list and wish to use that, then set `customStop` equal to **True**. If you do not have your own custom stopword list then set `customStop` equal to **False**.

Now we choose the language we will be using for the nltk stopwords list and the nltk stemmer. If you need a different language, simply change 'english' (keep the quotes) in the `language` variable to the anglicized name of the language you wish to use (e.g. 'spanish' instead of 'espanol' or 'german' instead of 'deutsch').

**NOTE: You can use both the nltk and custom stopword lists or you can use neither or just one or the other. You do NOT need to set them both to True or both to False. Use whatever works best for you.**

The `stopWords =[]` variable is simply an empty list. This is where the words from the nltk stopword list or your custom stopword list or both combined or neither (depending on what you decide) will reside later on. You do not need to do anything to this line of code.

The `tokenDict = {}` variable is an empty dictionary. This is where your documents will reside later. The file name for the document will be the key and the content of the document will be the value. This will be explained in more detail later. For now, you do not need to do anything to this line.

In [5]:
nltkStop = True
customStop = True
language = 'english'
stopWords = []
tokenDict = {}

### Stopwords
If you set `nltkStop` equal to **True** above then this will add the NLTK stopwords list to the empty list named `stopWords`.

You should have already chosen your desired language above, but if you wish to add any words to the stopWords list then add the word(s) you want as a stop word in the `stopWords.extend(['words', 'you', 'want', 'to', 'add'])` part of the code.

If you need to see the list of available languages in nltk simply remove the `#` from in front of the last line and run the cell. A list of available languages will print out.

In [6]:
if nltkStop is True:
    # NLTK Stop words
    stopWords = stopwords.words(language)

    stopWords.extend(['would', 'said', 'says', 'also'])
    #print (" ".join(stopwords.fileids()))

#### Add own stopword list

Here is where your own stopwords list is added if you set `customStop` equal to **True** above. Here you will need to change the folder names and file name to match your folders and file. Remember to put each folder name in quotes and in the correct order always putting the file name including the file extension (.txt) last.

In [7]:
if customStop is True:
    stopWordsFilepath = os.path.join(homePath, "Text-Analysis-master", "data", "earlyModernStopword.txt")

    with open(stopWordsFilepath, "r",encoding = 'utf-8') as f:
        stopWordsList = [x.strip() for x in f.readlines()]

    stopWords.extend(stopWordsList)

#### Dictionary and stemmer
Here we save the word stemmer we are using as the variable `stemmer`. 

The stemmer will truncate endings of words that end in 'ing' or 's' or 'es' et cetera. This will help ensure that words such as 'reading' and 'read' are considered as the same word so that the concept of 'to read' is given equal consideration. You should have already chosen your desired language up above.

A list of languages available with the nltk SnowballStemmer can be found by removing the `#` infront of the last line of code. 

In [8]:
stemmer = SnowballStemmer(language)
#print(" ".join(SnowballStemmer.languages))

### Functions
We need to create a function in order to stem and tokenize our data. Any time you see `def` that means we are **DE**claring a **F**unction. The `def` is usually followed by the name of the function being created and then in parentheses are the parameters required by the function. After the parentheses is a colon, which closes the declaration, then a bunch of code below which is indented. The indented code is the program statement or statements to be executed. Once you have created your function all you need to do in order to run it is call the function by name and make sure you have included all the required parameters in the parentheses. This allows you to call the function without having to write out all the code in the function every time you wish to perform that task.

#### Stemming and tokenization functions
Here we create two functions. The first is where we will use the `stemmer` variable above to create a function that will stem the words in our documents when applied.

The second tokenizes our documents which means it splits it into individual words. This function uses the `stemTokens` function as part of it so later in the code we will only need to apply the `tokenize` function. 

You should not need to make any changes to this block of code.

In [9]:
def stemTokens(tokens, stemmer):
    stemmed = []
    for item in tokens:
        stemmed.append(stemmer.stem(item))
    return stemmed

def tokenize(text):
    tokens = nltk.word_tokenize(text)
    stems = stemTokens(tokens, stemmer)
    return stems

#### Read in documents
Now we read in our documents and also perform some text cleaning. This code lower cases all the words as well as removes punctuation. Then it adds the file names and cleaned content of each file to our previously empty `tokenDict` dictionary above. You should not need to make any changes to this code.

A dictionary is similar to a list except it has what are called 'keys' and 'values'. This basically allows us to label our data. In this case we will be making the file names of our documents the 'keys' and the content of the file the 'values' so that each document name correlates to the content of that document.

In [10]:
for subdir, dirs, files in os.walk(dataHome):
    for file in files:
        if file.startswith('.'):
                continue
        filePath = subdir + os.path.sep + file
        with open(filePath, 'r', encoding = 'ISO-8859-1') as textFile:
            text = textFile.read()
            lowers = text.lower()
            noPunctuation = lowers.translate(str.maketrans('','', string.punctuation))
            tokenDict[file] = noPunctuation

Let's check and see if our dictionary now has our data. We are asking to see the first 10 keys of our dictionary.

In [11]:
print(list(tokenDict.keys())[0:10])

['1589TwoGentlemenOfVerona.txt', '1590TamingOfTheShrew.txt', '1591KingHenry6_1.txt', '1591KingHenry6_2.txt', '1591KingHenry6_3.txt', '1591TitusAndronicus.txt', '1592KingRichard3.txt', '1594ComedyOfErrors.txt', '1594LovesLaboursLost.txt', '1595KingRichard2.txt']


#### Tfidf Vectorizer

Here we weight the importance of each word in the document. This is done using Term Frequency-Inverse Document Frequency (Tfidf). This considers how important a word is based on the frequency in the whole corpus as well as in individual documents. This allows for words that might not have a high frequency in an entire collection, but do have a high frequency in one or two documents when compared to other words to still be given a higher level of importance throughout the text.

In [12]:
vectorizer = TfidfVectorizer(tokenizer = tokenize, stop_words=stopWords)
dtm = vectorizer.fit_transform(tokenDict.values())
testDF = pd.DataFrame(dtm.toarray(), index=tokenDict.keys(), columns = vectorizer.get_feature_names())
testDF = testDF.sort_index(axis = 0)

/N/u/cyberdh/Carbonate/dhPyEnviron/lib/python3.6/site-packages/sklearn/feature_extraction/text.py:300: UserWarning: Your stop_words may be inconsistent with your preprocessing. Tokenizing the stop words generated tokens ["'d", "'s", 'abov', 'afterward', 'alon', 'alreadi', 'alway', 'ani', 'anoth', 'anyon', 'anyth', 'anywher', 'becaus', 'befor', 'besid', 'dure', 'els', 'elsewher', 'everi', 'everyon', 'everyth', 'everywher', 'fifi', 'forti', 'henc', 'hereaft', 'herebi', 'howev', 'hundr', 'inde', 'mani', 'meanwhil', 'moreov', "n't", 'need', 'nobodi', 'noon', 'noth', 'nowher', 'onc', 'onli', 'otherwis', 'ourselv', 'perhap', 'pleas', 'say', 'sever', 'sha', 'sinc', 'sixti', 'someon', 'someth', 'sometim', 'somewher', 'themselv', 'thenc', 'thereaft', 'therebi', 'therefor', 'thi', 'togeth', 'twelv', 'twenti', 'veri', 'whatev', 'whenc', 'whenev', 'wherea', 'whereaft', 'wherebi', 'wherev', 'whi', 'wo', 'yourselv'] not in stop_words.
  'stop_words.' % sorted(inconsistent))


Now let's take a look at our data frame where the rows are the documents and the columns are the terms. To adjust which part of the data frame you see in the output change the numbers in the square brackets. To see the 10th thru the 20th row and the 100th to 110th column the numbers in the brackets should read `[10:20, 100:110]`. So the first set of numbers is for what rows you wish to see and the second set dictate the columns. 

In [13]:
testDF.iloc[:10, 7640:7650]

,massacr,massi,mast,master,mastercord,masterdom,mastergunn,masterleav,masterless,masterpiec
1589TwoGentlemenOfVerona.txt,0.000000,0.0,0.000000,0.079115,0.0,0.0,0.000000,0.0,0.0,0.0
1590TamingOfTheShrew.txt,0.000000,0.0,0.000000,0.104583,0.0,0.0,0.000000,0.0,0.0,0.0
1591KingHenry6_1.txt,0.015566,0.0,0.000000,0.012101,0.0,0.0,0.006773,0.0,0.0,0.0
1591KingHenry6_2.txt,0.000000,0.0,0.000000,0.051486,0.0,0.0,0.000000,0.0,0.0,0.0
1591KingHenry6_3.txt,0.000000,0.0,0.006464,0.010712,0.0,0.0,0.000000,0.0,0.0,0.0
1591TitusAndronicus.txt,0.009424,0.0,0.000000,0.006279,0.0,0.0,0.000000,0.0,0.0,0.0
1592KingRichard3.txt,0.008609,0.0,0.003462,0.012907,0.0,0.0,0.000000,0.0,0.0,0.0
1594ComedyOfErrors.txt,0.000000,0.0,0.013904,0.146888,0.0,0.0,0.000000,0.0,0.0,0.0
1594LovesLaboursLost.txt,0.000000,0.0,0.000000,0.059978,0.0,0.0,0.000000,0.0,0.0,0.0
1595KingRichard2.txt,0.000000,0.0,0.000000,0.003632,0.0,0.0,0.000000,0.0,0.0,0.0


The below code outputs all of the words that make up the columns once we have broken our corpus down into a Tfidf matrix.

In [14]:
# Get words that correspond to each column
vectorizer.get_feature_names()

['2d',
 '2s',
 '4d',
 '5s',
 '6d',
 '8d',
 'aaron',
 'abandon',
 'abas',
 'abash',
 'abat',
 'abatfowl',
 'abbess',
 'abbey',
 'abbot',
 'abbrevi',
 'abc',
 'abe',
 'abel',
 'abergavenni',
 'abet',
 'abhor',
 'abhorredst',
 'abhorson',
 'abi',
 'abid',
 'abil',
 'abird',
 'abject',
 'abjur',
 'abl',
 'ableed',
 'abler',
 'aboard',
 'abod',
 'abomin',
 'abort',
 'abound',
 'abov',
 'abraham',
 'abram',
 'abreast',
 'abreed',
 'abrew',
 'abridg',
 'abroach',
 'abroad',
 'abrog',
 'abrupt',
 'absenc',
 'absent',
 'abseybook',
 'absolut',
 'absolv',
 'abstain',
 'abstemi',
 'abstin',
 'abstract',
 'absurd',
 'absyrti',
 'abund',
 'abus',
 'abut',
 'abysm',
 'academ',
 'acapr',
 'accent',
 'accept',
 'access',
 'accessori',
 'accid',
 'accident',
 'accit',
 'acclam',
 'accommod',
 'accompani',
 'accomplic',
 'accomplish',
 'accord',
 'accost',
 'account',
 'accountst',
 'accouter',
 'accoutr',
 'accru',
 'accumul',
 'accurs',
 'accus',
 'accustom',
 'ace',
 'ach',
 'acheron',
 'achiev',
 'a

#### Run SVD and Cosine Similarity

Here we run our Tfidf matrix created above through Singular Value Decomposition and then calculate the Cosine Similarity of the documents to one another. 

Singular Value Decomposition condenses our Tfidf matrix down a bit to make it easier to process. Here we also set the number of dimensions (`n_components`) , how many times we iterate over the corpus (`n_iter`), and then set the seed (`random_state`) so that the results are reproducable. At the moment the `random_state` is set to 42 which sets the seed for the random number generator, but feel free to adjust the number to get a slightly different output. Just make sure you keep the seed the same once you find one you like for reproducibility.

Cosine similarity is where we measure how similar the documents are to one another. The result is a number between -1 and 1 with 1 being a perfect match (which we will get when the document is compared to itself) and -1 being completely different which we might get if we have a document of all numbers and one of all words with no numbers at all. Usually, even documents that are about unrelated topics share some common words and so are not completely dissimilar.

In [15]:
lsa = TruncatedSVD(n_components = 100, n_iter = 1000, random_state = 42)
dtmLsa = lsa.fit_transform(dtm)
cosineSim = cosine_similarity(dtmLsa)

#### Save as a data frame

Now we save the results as a data frame. First we name the output .csv file so it matches our data. We do this in the first line of the cell.

Next we add color based on the numbers in the data frame cells, so right now we have it set so that the more dissimilar the more red the cell will be and the more similar the more green it will be with those in the middle looking more yellow. You can adjust the colors by changing the `colorChoice` variable to something else. Options for colors can be found here: [https://matplotlib.org/tutorials/colors/colormaps.html](https://matplotlib.org/tutorials/colors/colormaps.html) 

Then we create the data frame and say we want the rows and columns to be labeled with the file names. Then we sort the columns in alphanumeric order by column header, then we sort the rows alphanumericaly by row label.  

In [16]:
csvFileName = "docSimilarityMatrix.csv"
colorChoice = 'RdYlGn'

df = pd.DataFrame(cosineSim, index = tokenDict.keys(), columns=tokenDict.keys())
dfS = df[sorted(df)]
sortedDf = dfS.sort_index(axis = 0)
sortedDf.to_csv(os.path.join(dataResults, csvFileName))
#colorDf = sortedDf.style.background_gradient(cmap=colorChoice)
#colorDf

### Plot Heatmap

Now we plot a heatmap and save it as a .png file for use in a paper, presentation, or even a website. 

First, we name the output file of the heatmap which is in the variable `heatmapFileName`. 

Next we choose the dpi level we want the output to be. We have it set to 300 which is generally good for both print and website use. 

Then we choose the color scheme we want. At the moment we have one where green represents the highest number (most similar) and red represents the lowest numbers (least similar) with yellow being more in the middle. If another color scheme would work better for you other options for color schemes can be found here: [https://matplotlib.org/tutorials/colors/colormaps.html](https://matplotlib.org/tutorials/colors/colormaps.html)

Now we choose the font size. This will depend on your data and the size of your heatmap, so you'll want to play around with the number in `fontScale` until you find a value that works with your heatmap. 


Now we plot our heatmap!

In [17]:
#Variables
#heatmapFileName = 'DocSimHeatmap.svg'
#dpi = 300
#colorScheme = 'RdYlGn'
#fontScale = 5

# Plot
#figureSize = len(sortedDf)
#sns.set(rc={'figure.figsize':(figureSize + 10, figureSize)}, font_scale = fontScale)
#ax = sns.heatmap(sortedDf, cmap = colorScheme)
#ax.figure.savefig(os.path.join(dataResults, heatmapFileName), dpi = dpi, bbox_inches='tight')
#plt.show()

## VOILA!!

This notebook was adapted from https://www.datascienceassn.org/sites/default/files/users/user1/lsa_presentation_final.pdf at Colorado University, Boulder. Accessed on 02/01/2019.